In [3]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [16]:
os.getcwd()

'/Users/juank/Documents/Flatiron/phase_3/chicago_car_crashes/notebooks'

In [4]:
crashes = pd.read_csv('../data/crashes_cleaned.csv')

In [5]:
target = crashes['PRIM_CONTRIBUTORY_CAUSE']

target_transform = LabelEncoder()

crashes['PRIM_CONTRIBUTORY_CAUSE'] = target_transform.fit_transform(crashes['PRIM_CONTRIBUTORY_CAUSE'])

In [7]:
crashes.drop('Unnamed: 0', axis = 1, inplace = True)

# Logistic Regression 

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_validate

In [10]:
X = crashes.loc[:, (crashes.columns != 'PRIM_CONTRIBUTORY_CAUSE') & (crashes.columns != 'CRASH_RECORD_ID')]
y = crashes.loc[:, 'PRIM_CONTRIBUTORY_CAUSE']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state = 42)

scaler = StandardScaler()
X_tf_sc = scaler.fit_transform(X_train)

X_test_tr = scaler.transform(X_test)

log_reg = LogisticRegression(solver = 'sag')
log_reg.fit(X_tf_sc, y_train)

LogisticRegression(solver='sag')

In [19]:
pred = log_reg.predict(X_test_tr)
pred_proba = log_reg.predict_proba(X_test_tr)

log_reg.score(X_test_tr, y_test)

0.4572655690765927

In [27]:
log_reg_cv = LogisticRegression(random_state = 42,  solver = 'sag')

cv_results = cross_validate(log_reg_cv, X_train, y_train, cv = 5)

In [ ]:
cv_results

In [ ]:
C_list = [1e-4,1e-3, 1e-2, 1e-1, 1, 10, 100, 1e3]
cv_scores = []
cv_scores_std = []

for c in C_list:
    logreg = LogisticRegression(C = c, solver = 'sag', random_state = 42) 
    cv_loop_results = cross_validate(
                X=X_tf_sc, 
                y=y_train,
                estimator=logreg, 
                cv=5)
    cv_scores.append(np.mean(cv_loop_results['test_score']))
    cv_scores_std.append(np.std(cv_loop_results['test_score']))